# 환경설정

In [10]:
# 설치(import)
from paddleocr import PaddleOCRVL

# PaddlePaddle/PaddleOCR

In [12]:
# 모델 초기화
pipeline = PaddleOCRVL()

image_path = r'C:\ddddwoo\project\project2\AiMind-AiModels\test.png'
result = pipeline.predict(image_path)

# 결과 확인
for res in result:
    res.print() # 결과를 예쁘게 출력해줍니다.

RuntimeError: A dependency error occurred during pipeline creation. Please refer to the installation documentation to ensure all required dependencies are installed.